In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wall-street-journal-2/WSJ_24.pos
/kaggle/input/wall-street-journal-2/test.words
/kaggle/input/wall-street-journal-2/hmm_vocab.txt
/kaggle/input/wall-street-journal-2/WSJ_02-21.pos


In [2]:
import string
from collections import defaultdict

In [3]:
with open("/kaggle/input/wall-street-journal-2/WSJ_02-21.pos") as f:
    train_corpus = f.readlines()

train_corpus[:5]

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']

In [4]:
with open("/kaggle/input/wall-street-journal-2/hmm_vocab.txt", 'r') as f:
    voc = f.read().split('\n')

print(voc[:50],'\n')
print(voc[-50:])

['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15'] 

['yards', 'yardstick', 'year', 'year-ago', 'year-before', 'year-earlier', 'year-end', 'year-on-year', 'year-round', 'year-to-date', 'year-to-year', 'yearlong', 'yearly', 'years', 'yeast', 'yelled', 'yelling', 'yellow', 'yen', 'yes', 'yesterday', 'yet', 'yield', 'yielded', 'yielding', 'yields', 'you', 'young', 'younger', 'youngest', 'youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{', '}', '']


In [5]:
voc.sort()
vocab={}

for i,word in enumerate(voc):
    vocab[word] = i

cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt>20: break

:0
!:1
#:2
$:3
%:4
&:5
':6
'':7
'40s:8
'60s:9
'70s:10
'80s:11
'86:12
'90s:13
'N:14
'S:15
'd:16
'em:17
'll:18
'm:19
'n':20


In [6]:
with open("/kaggle/input/wall-street-journal-2/WSJ_24.pos",'r') as f:
    y = f.readlines()

y[:10]

['The\tDT\n',
 'economy\tNN\n',
 "'s\tPOS\n",
 'temperature\tNN\n',
 'will\tMD\n',
 'be\tVB\n',
 'taken\tVBN\n',
 'from\tIN\n',
 'several\tJJ\n',
 'vantage\tNN\n']

In [7]:
def assign_unk(word):
    punct = string.punctuation
    noun_suffs = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
    verb_suffs = ["ate", "ify", "ise", "ize"]
    adj_suffs = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
    adv_suffs = ["ward", "wards", "wise"]
    
    if any(char.isdigit() for char in word):
        return "--unk_digit--"

    elif any(char in punct for char in word):
        return "--unk_punct--"

    elif any(char.isupper() for char in word):
        return "--unk_upper--"

    elif any(word.endswith(suff) for suff in noun_suffs):
        return "--unk_noun--"

    elif any(word.endswith(suff) for suff in verb_suffs):
        return "--unk_verb"

    elif any(word.endswith(suff) for suff in adj_suffs):
        return "--unk_adj--"

    elif any(word.endswith(suff) for suff in adv_suffs):
        return "--unk_adv--"

    return "unk--"
    

In [8]:
def preprocess(vocab,data):
    orig = []
    prep = []

    with open(data, 'r') as f:
        for word in f:
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data, "r").readlines()))
    assert(len(prep) == len(open(data, "r").readlines()))

    return orig, prep

In [9]:
_, prep = preprocess(vocab,"/kaggle/input/wall-street-journal-2/test.words")   

In [10]:
print(len(prep))
prep[:10]

34199


['The',
 'economy',
 "'s",
 'temperature',
 'will',
 'be',
 'taken',
 'from',
 'several',
 'unk--']

In [11]:
def get_word_tag(line, vocab):
    if not(line.split()):
        word = "--n--"
        tag = "--s--"
    else:
        word,tag = line.split()
        if word not in vocab:
            word = assign_unk(word)
    return word, tag

**POS Tagging**

In [12]:
def create_dicts(train_corpus, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    prev_tag = '--s--'
    i=0

    for word_tag in train_corpus:
        i+=1
        
        if i%50000 == 0:
            print(f"word count = {i}")
        word, tag = get_word_tag(word_tag, vocab)

        transition_counts[(prev_tag, tag)]+=1
        emission_counts[(tag, word)]+=1
        tag_counts[tag]+=1

        prev_tag = tag

    return transition_counts, emission_counts, tag_counts
    

In [13]:
transition_counts, emission_counts, tag_counts = create_dicts(train_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000
word count = 400000
word count = 450000
word count = 500000
word count = 550000
word count = 600000
word count = 650000
word count = 700000
word count = 750000
word count = 800000
word count = 850000
word count = 900000
word count = 950000


In [14]:
states = sorted(tag_counts.keys())
print("Number of POS Tags (states): ", len(states))
print(states)

Number of POS Tags (states):  46
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [15]:
list(transition_counts.items())[:3]

[(('--s--', 'IN'), 5050), (('IN', 'DT'), 32364), (('DT', 'NNP'), 9044)]

In [16]:
list(emission_counts.items())[:3]

[(('IN', 'In'), 1735), (('DT', 'an'), 3142), (('NNP', 'Oct.'), 317)]

In [17]:
for tup,cnt in emission_counts.items():
    if tup[1]=='back': print(tup,cnt)


('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


In [18]:
def predict_pos(prep, y, emission_counts, vocab, states):
    num_crct = 0
    total = len(y)

    all_words = set(emission_counts.keys())
    
    for word, y_tup in zip(prep,y):
        y_tup_l = y_tup.split()
        if len(y_tup_l)==2:
            true_label = y_tup_l[1]
        else:
            continue

        count_final = 0
        pos_final = ''

        if word in vocab:
            for pos in states:
                key = (pos,word)

                if key in emission_counts:
                    count = emission_counts[key]
                    if count>count_final:
                        count_final = count
                        pos_final = pos

            if pos_final == true_label:
                num_crct+=1

    accuracy = num_crct/total
    return accuracy

In [19]:
accuracy = predict_pos(prep, y, emission_counts, vocab, states)
accuracy

0.8858446153396298

**Hidden Markov Models for POS**

In [20]:
def create_trans_mat(transition_counts, tag_counts, alpha):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)

    A = np.zeros((num_tags,num_tags))

    for i in range(num_tags):
        for j in range(num_tags):
            a = all_tags[i]
            b = all_tags[j]
            count = transition_counts.get((a,b),0)
            count_p = tag_counts[a]
            A[i][j] = (count+alpha)/(count_p+alpha*num_tags)
    return A
    

In [21]:
A = create_trans_mat(transition_counts, tag_counts, 0.001)

In [22]:
A[3][1]

0.16910191896905374

In [23]:
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns=states[30:35])
A_sub

,RBS,RP,SYM,TO,UH
RBS,2.217069e-06,2.217069e-06,2.217069e-06,0.008870,2.217069e-06
RP,3.756509e-07,7.516775e-04,3.756509e-07,0.051089,3.756509e-07
SYM,1.722772e-05,1.722772e-05,1.722772e-05,0.000017,1.722772e-05
TO,4.477336e-05,4.472863e-08,4.472863e-08,0.000090,4.477336e-05
UH,1.030439e-05,1.030439e-05,1.030439e-05,0.061837,3.092348e-02


In [24]:
def create_emi_mat(emission_counts, tag_counts, vocab, alpha):
    num_tags = len(tag_counts)
    all_tags = sorted(tag_counts.keys())
    num_words = len(vocab)

    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        for j in range(num_words):
            a = all_tags[i]
            b = vocab[j]

            count = emission_counts.get((a,b),0)
            count_t = tag_counts[a]
            B[i][j] = (count+alpha)/(count_t+alpha*num_words)
    return B

In [25]:
B = create_emi_mat(emission_counts, tag_counts, list(vocab), 0.001)

In [26]:
B[0][0]

6.032199882975323e-06

In [27]:
cidx = ['725','adroitly','engineers', 'promoted', 'synergy']
cols = [vocab[a] for a in cidx]

rvals =['CD','NN','NNS', 'VB','RB','RP']
rows = [states.index(a) for a in rvals]

B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx)
B_sub

,725,adroitly,engineers,promoted,synergy
CD,8.201296e-05,2.732854e-08,2.732854e-08,2.732854e-08,2.732854e-08
NN,7.521128e-09,7.521128e-09,7.521128e-09,7.521128e-09,2.257091e-05
NNS,1.670013e-08,1.670013e-08,4.676203e-04,1.670013e-08,1.670013e-08
VB,3.779036e-08,3.779036e-08,3.779036e-08,3.779036e-08,3.779036e-08
RB,3.226454e-08,6.456135e-05,3.226454e-08,3.226454e-08,3.226454e-08
RP,3.723317e-07,3.723317e-07,3.723317e-07,3.723317e-07,3.723317e-07


In [28]:
states.index('CD')

10